    Let’s first implement linear regression in PyTorch and learn point estimates for the parameters w and b. 
    Then we’ll see how to incorporate uncertainty into our estimates by using Pyro to implement Bayesian linear regression.
    
    y=wX+b+ϵ
    ϵ represents observation noise
    Mainly learn weight and bias
    
    prior: 先見條件
    baysiean regression: exists prior (difference against linear regressiom)
     
    prior: depend on experience before, imagination about w and b 根據以前做的經驗，對於w, b的想像
    posterior: prior after learning 學習完之後的prior

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn

import pyro
from pyro.distributions import Normal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
# for CI testing
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

    We’ll generate a toy dataset with one feature and w=3 and b=1 as follows:

In [3]:
N = 100  # size of toy data

def build_linear_dataset(N, p=1, noise_std=0.01):
    X = np.random.rand(N, p)
    # w = 3
    w = 3 * np.ones(p)
    # b = 1
    y = np.matmul(X, w) + np.repeat(1, N) + np.random.normal(0, noise_std, size=N)
    y = y.reshape(N, 1)
    # torch.tensor 要先變成tensor的形式
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X, y), 1)
    assert data.shape == (N, p + 1)
    return data

### Regression

In [4]:
class RegressionModel(nn.Module):
    def __init__(self, p):
        # p = number of features
        super(RegressionModel, self).__init__()
        
        # return a model
        self.linear = nn.Linear(p, 1)
    
    # 哪邊有 nonlinearity?
    def forward(self, x):
        return self.linear(x)
    
# number of feature
regression_model = RegressionModel(1)

In [5]:
loss_fn = torch.nn.MSELoss(size_average=False)
optim = torch.optim.Adam(regression_model.parameters(), lr=0.05)
num_iterations = 1000 if not smoke_test else 2

def main():
    data = build_linear_dataset(N)
    x_data = data[:, :-1]
    y_data = data[:, -1]
    for j in range(num_iterations):
        # run the model forward on the data
        y_pred = regression_model(x_data).squeeze(-1)
        # calculate the mse loss
        loss = loss_fn(y_pred, y_data)
        # initialize gradients to zero
        optim.zero_grad()
        # backpropagate
        loss.backward()
        # take a gradient step
        optim.step()
        if (j + 1) % 50 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))
    # Inspect learned parameters
    print("Learned parameters:")
    for name, param in regression_model.named_parameters():
        print("%s: %.3f" % (name, param.data.numpy()))

if __name__ == '__main__':
    main()

[iteration 0050] loss: 27.8627
[iteration 0100] loss: 12.8889
[iteration 0150] loss: 5.1976
[iteration 0200] loss: 1.7172
[iteration 0250] loss: 0.4739
[iteration 0300] loss: 0.1142
[iteration 0350] loss: 0.0290
[iteration 0400] loss: 0.0124
[iteration 0450] loss: 0.0097
[iteration 0500] loss: 0.0093
[iteration 0550] loss: 0.0093
[iteration 0600] loss: 0.0093
[iteration 0650] loss: 0.0093
[iteration 0700] loss: 0.0093
[iteration 0750] loss: 0.0093
[iteration 0800] loss: 0.0093
[iteration 0850] loss: 0.0093
[iteration 0900] loss: 0.0093
[iteration 0950] loss: 0.0093
[iteration 1000] loss: 0.0093
Learned parameters:
linear.weight: 3.005
linear.bias: 0.998


## Bayesian Regression

    In order to do this, we’ll ‘lift’ the parameters w and b to random variables. We can do this in Pyro via random_module()
    which effectively takes a given nn.Module and turns it into a distribution over the same module
    
    Need "prior" !!!

In [6]:
# 為什麼這邊是用二維的？
# 1x1 --> 就是一個值
loc = torch.zeros(1, 1)
scale = torch.ones(1, 1)
# define a unit normal prior
prior = Normal(loc, scale)

# overload the parameters in the regression module with samples from the prior
lifted_module = pyro.random_module("regression_module", regression_model, prior)
# sample a regressor from the prior
sampled_reg_model = lifted_module()

In [7]:
def model(data):
    # Create unit normal priors over the parameters
    # N(μ=0,σ=10)
    loc, scale = torch.zeros(1, 1), 10 * torch.ones(1, 1)
    bias_loc, bias_scale = torch.zeros(1), 10 * torch.ones(1)
    w_prior = Normal(loc, scale).independent(1)
    b_prior = Normal(bias_loc, bias_scale).independent(1)
    priors = {'linear.weight': w_prior, 'linear.bias': b_prior}
    # lift module parameters to random variables sampled from the priors
    # prior: p(z) regression_model: p(x/z)
    # pyro.random_module is like a constructor
    lifted_module = pyro.random_module("module", regression_model, priors)
    
    # inference
    # !!! sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    
    with pyro.iarange("map", N):
        x_data = data[:, :-1]
        y_data = data[:, -1]

        # run the regressor forward conditioned on data
        prediction_mean = lifted_reg_model(x_data).squeeze(-1)
        
        # condition on the observed data
        pyro.sample("obs",
                    Normal(prediction_mean, 0.1 * torch.ones(data.size(0))),
                    obs=y_data)

In [8]:
softplus = torch.nn.Softplus()

# data 一定要寫，但這邊不會用到，標準格式
def guide(data):
    # define our variational parameters
    w_loc = torch.randn(1, 1)
    # note that we initialize our scales to be pretty narrow
    w_log_sig = torch.tensor(-3.0 * torch.ones(1, 1) + 0.05 * torch.randn(1, 1))
    b_loc = torch.randn(1)
    b_log_sig = torch.tensor(-3.0 * torch.ones(1) + 0.05 * torch.randn(1))
    # register learnable params in the param store
    mw_param = pyro.param("guide_mean_weight", w_loc)
    sw_param = softplus(pyro.param("guide_log_scale_weight", w_log_sig))
    mb_param = pyro.param("guide_mean_bias", b_loc)
    sb_param = softplus(pyro.param("guide_log_scale_bias", b_log_sig))
    # guide distributions for w and b
    w_dist = Normal(mw_param, sw_param).independent(1)
    b_dist = Normal(mb_param, sb_param).independent(1)
    dists = {'linear.weight': w_dist, 'linear.bias': b_dist}
    # overload the parameters in the module with random samples
    # from the guide distributions
    lifted_module = pyro.random_module("module", regression_model, dists)
    # sample a regressor (which also samples w and b)
    return lifted_module()

## Inference

In [10]:
optim = Adam({"lr": 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [11]:
def main():
    pyro.clear_param_store()
    data = build_linear_dataset(N)
    for j in range(num_iterations):
        # calculate the loss and take a gradient step
        loss = svi.step(data)
        if j % 100 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss / float(N)))

if __name__ == '__main__':
    main()

[iteration 0001] loss: 387.1771
[iteration 0101] loss: 2.6335
[iteration 0201] loss: 0.0786
[iteration 0301] loss: -1.0272
[iteration 0401] loss: -1.2048
[iteration 0501] loss: -1.2476
[iteration 0601] loss: -1.2592
[iteration 0701] loss: -1.2531
[iteration 0801] loss: -1.2487
[iteration 0901] loss: -1.2363


    Baysiean regression學到了 w跟b 也是一種分布
    學到w 跟 b 是從哪種distribution出來的
    想像一條平面上有很多條線，sample就是取一條線

In [12]:
for name in pyro.get_param_store().get_all_param_names():
    print("[%s]: %.3f" % (name, pyro.param(name).data.numpy()))

[guide_log_scale_weight]: -3.780
[guide_mean_weight]: 2.999
[guide_log_scale_bias]: -4.149
[guide_mean_bias]: 1.000


In [13]:
X = np.linspace(6, 7, num=20)
y = 3 * X + 1
X, y = X.reshape((20, 1)), y.reshape((20, 1))
x_data, y_data = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
loss = nn.MSELoss()
y_preds = torch.zeros(20, 1)
for i in range(20):
    # guide does not require the data
    sampled_reg_model = guide(None)
    # run the regression model and add prediction to total
    y_preds = y_preds + sampled_reg_model(x_data)
# take the average of the predictions
y_preds = y_preds / 20
print ("Loss: ", loss(y_preds, y_data).item())

Loss:  0.0005512561183422804
